In [45]:
# coding=utf-8
import pandas as pd

import numpy as np
print(np._version)

<module 'numpy._version' from 'C:\\ProgramData\\Anaconda3\\envs\\py39\\lib\\site-packages\\numpy\\_version.py'>


In [ ]:
from datetime import datetime

# 加载cvs文件
"""
TestTime.csv文件：
"name","time","date"
'Bob',21:33:30,2019-10-10
'Jerry',21:30:15,2019-10-10
'Tom',21:25:30,2019-10-10
'Vince',21:20:10,2019-10-10
'Hank',21:40:15,2019-10-10

指定parse_dates = [ ['time', 'date'] ]，即将[ ['time', 'date'] ]两列的字符串先合并后解析方可。合并后的新列会以下划线'_'连接原列名命名
本例中解析后的命名为：time_date,解析得到的日期格式列会作为DataFrame的第一列。

如果写成了parse_dates=['time', 'date'] ，pd.read_csv()会分别对'time', 'date'进行字符串转日期，此外还会造成一个小小的麻烦。
由于本例中的Time时间列格式为'HH:MM:SS'，
parse_dates默认调用dateutil.parser.parse解析为Datetime格式，在解析time这一列时，会自作主张在前面加上一个当前日期。
结果：
      name                time         date
0    'Bob' 2019-10-17 21:33:30   2019-10-10
1  'Jerry' 2019-10-17 21:30:15   2019-10-10
2    'Tom' 2019-10-17 21:25:30   2019-10-10
3  'Vince' 2019-10-17 21:20:10   2019-10-10
4   'Hank' 2019-10-17 21:40:15   2019-10-10

"""

In [24]:
df = pd.read_csv('./data/EURUSD-100.txt',
                 parse_dates=['<DTYYYYMMDD>', '<TIME>'])
print(df.head(3))
df.dtypes

  <TICKER> <DTYYYYMMDD>  <TIME>  <OPEN>  <HIGH>   <LOW>  <CLOSE>  <VOL>
0   EURUSD   2001-01-02  230100  0.9507  0.9507  0.9507   0.9507      4
1   EURUSD   2001-01-02  230200  0.9506  0.9506  0.9505   0.9505      4
2   EURUSD   2001-01-02  230300  0.9505  0.9507  0.9505   0.9506      4


<TICKER>                object
<DTYYYYMMDD>    datetime64[ns]
<TIME>                  object
<OPEN>                 float64
<HIGH>                 float64
<LOW>                  float64
<CLOSE>                float64
<VOL>                    int64
dtype: object

In [28]:
df = pd.read_csv('./data/EURUSD-100.txt',
                 parse_dates=[['<DTYYYYMMDD>', '<TIME>']])
print(df.head(3))
df.dtypes

  <DTYYYYMMDD>_<TIME> <TICKER>  <OPEN>  <HIGH>   <LOW>  <CLOSE>  <VOL>
0 2001-01-02 23:01:00   EURUSD  0.9507  0.9507  0.9507   0.9507      4
1 2001-01-02 23:02:00   EURUSD  0.9506  0.9506  0.9505   0.9505      4
2 2001-01-02 23:03:00   EURUSD  0.9505  0.9507  0.9505   0.9506      4


<DTYYYYMMDD>_<TIME>    datetime64[ns]
<TICKER>                       object
<OPEN>                        float64
<HIGH>                        float64
<LOW>                         float64
<CLOSE>                       float64
<VOL>                           int64
dtype: object

In [29]:
# 将DATE设定为index 执行一遍就好了
df.set_index('<DTYYYYMMDD>_<TIME>', inplace=True)

In [38]:

rule_type = '1D'  # 周期单位: 5T=5分钟 1H=1小时 1D=1天 1W=1周
period_df = df.resample(rule=rule_type, base=0, label='left', closed='left').agg(
    {'<OPEN>': 'first',
     '<HIGH>': 'max',
     '<LOW>': 'min',
     '<CLOSE>': 'last',
     '<VOL>': 'sum'
     })

# period_df['<DTYYYYMMDD>'] = period_df['<DTYYYYMMDD>_<TIME>'].dt.date
print(period_df.head(3))

period_df = period_df[['<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>']]
# base参数：帮助确定转换周期开始的时间
# label='left', closed='left'，建议统一设置成'left'

# ===去除不必要的数据
# 去除一天都没有交易的周
period_df.dropna(subset=['<OPEN>'], inplace=True)
period_df


C:\Users\yalin\AppData\Local\Temp/ipykernel_25992/1385777546.py:2: FutureWarning: 'base' in .resample() and in Grouper() is deprecated.
The new arguments that you should use are 'offset' or 'origin'.

>>> df.resample(freq="3s", base=2)

becomes:

>>> df.resample(freq="3s", offset="2s")

  period_df = df.resample(rule=rule_type, base=0, label='left', closed='left').agg(


KeyError: '<DTYYYYMMDD>_<TIME>'

In [32]:
# 保存文件

df.to_csv('./data/EURUSD-1D.csv')


In [ ]:
# ===rule的取值
"""
    B       business day frequency
    C       custom business day frequency (experimental)
    D       calendar day frequency
    W       weekly frequency
    M       month end frequency
    SM      semi-month end frequency (15th and end of month)
    BM      business month end frequency
    CBM     custom business month end frequency
    MS      month start frequency
    SMS     semi-month start frequency (1st and 15th)
    BMS     business month start frequency
    CBMS    custom business month start frequency
    Q       quarter end frequency
    BQ      business quarter endfrequency
    QS      quarter start frequency
    BQS     business quarter start frequency
    A       year end frequency
    BA      business year end frequency
    AS      year start frequency
    BAS     business year start frequency
    BH      business hour frequency
    H       hourly frequency
    T       minutely frequency
    S       secondly frequency
    L       milliseonds
    U       microseconds
    N       nanoseconds
"""
